<h1><center>Fundamentals of Q learning 

## Before we start, let's learn about gym

    * Gym is an OpenAI platform which provides easy access to various environments. 
    * Environment (of a game) will have attributes such as `action_space`, `observation_space` & `reward_range`
    * We use `step()` method to advance environment when a particular action is chosen
    * `step()` returns four outputs; observation, reward, done, info
        > Observation is `Discrete`/ `Box` object which gives info about what state env is in currently 
        > Reward is what we achieved after action
        > Done will be true if the env reaches its end state
        > Info contains metadata
        
Read more here: https://gym.openai.com/docs 


____________________________

## Now a brief about RL 

Reinforced Learning is all about learning what chain of actions to follow to go through a certain chain of states in order to reach a final state in a way that maximizes our **total** reward

### Terminology
1. Environment
2. Agent
3. State
4. Observation
5. Action
6. Reward
7. Environment modelling by an Agent
8. Policy of an Agent
9. Value function of an Agent
    * Value function for a fixed action ($\pi$) for each state $(S)$ : $Q_{\pi}(S_i)$
    * Value function for variable actions for each state $(S)$ : $Q(a_i, S_i)$

# The whole drama is for learning $Q(a_i, S_i)$

The simplest form of representing this function is using a table. 
A table of dimension $(#states, #actions)$. Each cell in the table denotes the value of being in a state $s$ and taking an action $a$ in $s$

## Frozen lake env in Gym
To establish and learn that kind of Q-table, Frozen lake env in Gym would be best fit.
This env is 4x4 grid, each cell can be a stable block, a hole or a goal position. 
Reward to be in any cell except goal cell is zero. Goal cell carries a reward of one.

Now possible states in this env is one of the cells, so, 16.
Similarly possible actions (left, right, top, bottom) are 4.
Let's dive.


In [1]:
import gym
import numpy as np

env = gym.make('FrozenLake-v0')

[2017-07-21 01:58:51,712] Making new env: FrozenLake-v0


In [2]:
print "#states =", env.observation_space.n
print "#actions =", env.action_space.n

#states = 16
#actions = 4


In [21]:
# Init Q table (Value function)

Q = np.zeros([env.observation_space.n, env.action_space.n])

# Now train the env by processing rewards and learn the Q function
episodes = 2000
learning_rate = 0.80
discount_fact = 0.95
rewards_over_episodes = []

# Play #episode games
for i in range(1, episodes):
    accumulated_reward_for_episode = 0
    done = False
    s = env.reset()  # Reset and get default/ init state.
    
    for _ in range(99):
        
        # We are learning now, greedily pick an action while being in state `s`
        action = np.argmax(Q[s, :] + (np.random.randn(1, env.action_space.n)*(1./(1 + i))))

        # See the consequence of this action
        new_state, reward, done, _ = env.step(action)
        
        # Alright, we have received reward, let's remember what earned us that amount
        # Bellman equation & the fact that we want the updation to be steady and small
        Q[s, action] =   Q[s, action] + learning_rate * (reward + (discount_fact * np.max(Q[new_state, :])))
                       - learning_rate * Q[s, action]
        
        # Updating iteration vals
        accumulated_reward_for_episode += reward
        s = new_state
        
        # Episode ended
        if done is True:
            break
            
    rewards_over_episodes.append(accumulated_reward_for_episode)
    if i % 100 == 0:
        print "Reward @ episode %d = %.2f" % (i, sum(rewards_over_episodes)/episodes)
    
print "TRAINING ENDED"
    

Reward @ episode 100 = 0.00
Reward @ episode 200 = 0.02
Reward @ episode 300 = 0.04
Reward @ episode 400 = 0.06
Reward @ episode 500 = 0.09
Reward @ episode 600 = 0.11
Reward @ episode 700 = 0.14
Reward @ episode 800 = 0.17
Reward @ episode 900 = 0.21
Reward @ episode 1000 = 0.24
Reward @ episode 1100 = 0.28
Reward @ episode 1200 = 0.31
Reward @ episode 1300 = 0.34
Reward @ episode 1400 = 0.37
Reward @ episode 1500 = 0.41
Reward @ episode 1600 = 0.45
Reward @ episode 1700 = 0.49
Reward @ episode 1800 = 0.52
Reward @ episode 1900 = 0.56
TRAINING ENDED


In [16]:
print Q

[[  2.43567931e-01   1.29953830e-02   1.37789988e-02   1.32270488e-02]
 [  7.30154553e-04   8.34075080e-04   3.19486029e-04   2.26955196e-01]
 [  2.92785670e-03   9.91106073e-02   4.55518449e-03   4.24503566e-03]
 [  6.06637322e-04   5.89402814e-04   0.00000000e+00   4.19002118e-02]
 [  3.60335085e-01   3.60273873e-03   8.06561456e-03   3.68616199e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.28962214e-04   1.14934259e-04   5.71909935e-02   4.39082815e-06]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   5.65256390e-03   4.06607033e-01]
 [  3.66698844e-03   2.41311287e-01   0.00000000e+00   0.00000000e+00]
 [  7.74143483e-01   2.15206745e-04   0.00000000e+00   1.26652371e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   7.30694621e-01   0.00000000e+00]
 [  0.

### All good, but what if the env is complex?
In a way, that it has waay too many states and waaay too many possible actions?

Nothing, there will be a bigger table.
But, we have a better method to represent that kind of a big table 

We will train a Neuralnet to learn this Value function by feeding reward converted as target Q values.
Let's do that.